In [6]:
import sqlite3, os

In [7]:
os.listdir("../visionUI/vhelio_holes/")


['candidates',
 'labels',
 'dataset.db',
 'images',
 'dataset.yaml',
 'labels.cache',
 'new_visionUI.db',
 'visionUI.db']

In [8]:
sqlite3.connect("../visionUI/vhelio_holes/dataset.db")

In [13]:
def explore_database(db_path):
    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    result = []

    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Explore tables and their columns
    for table in tables:
        table_name = table[0]
        result.append(f"\nTable: {table_name}")
        
        # Get column information for each table
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        
        for column in columns:
            result.append(f"  - {column[1]} ({column[2]})")

    # Close the connection
    conn.close()

    return "\n".join(result)

# Use the function and return the result
explore_database("../visionUI/vhelio_holes/visionUI.db")

'\nTable: annotations\n  - id (INTEGER)\n  - file_id (TEXT)\n  - type (INTEGER)\n  - x (NUMERIC)\n  - y (NUMERIC)\n  - x2 (NUMERIC)\n  - y2 (NUMERIC)\n  - class_id (INTEGER)\n  - confidence (NUMERIC)\n\nTable: sqlite_sequence\n  - name ()\n  - seq ()\n\nTable: classes\n  - id (INTEGER)\n  - name (TEXT)\n  - description (TEXT)\n  - encoding (INTEGER)\n  - type (INTEGER)'

In [14]:
print(_)


Table: annotations
  - id (INTEGER)
  - file_id (TEXT)
  - type (INTEGER)
  - x (NUMERIC)
  - y (NUMERIC)
  - x2 (NUMERIC)
  - y2 (NUMERIC)
  - class_id (INTEGER)
  - confidence (NUMERIC)

Table: sqlite_sequence
  - name ()
  - seq ()

Table: classes
  - id (INTEGER)
  - name (TEXT)
  - description (TEXT)
  - encoding (INTEGER)
  - type (INTEGER)


In [3]:
import sqlite3
import os
from PIL import Image
import io
from tqdm import tqdm
import concurrent.futures

def process_image(file_path):
    with Image.open(file_path) as img:
        width, height = img.size
        # Redimensionner l'image si elle est trop grande
        if width > 1024 or height > 1024:
            img.thumbnail((1024, 1024))
        # Convertir en JPEG au lieu de PNG pour une meilleure compression
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='JPEG', quality=85)
        return {
            'data': img_byte_arr.getvalue(),
            'width': img.width,
            'height': img.height
        }

def import_data(db_path, image_dirs):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    data = {
        'annotations': [],
        'classes': {},
        'images': {}
    }

    print("Importing classes...")
    cursor.execute("SELECT * FROM classes")
    for row in tqdm(cursor.fetchall()):
        data['classes'][row[0]] = {
            'name': row[1],
            'description': row[2],
            'encoding': row[3],
            'type': row[4]
        }

    print("Importing annotations...")
    cursor.execute("SELECT * FROM annotations")
    data['annotations'] = [dict(zip(['id', 'file_id', 'type', 'x', 'y', 'x2', 'y2', 'class_id', 'confidence'], row)) 
                           for row in tqdm(cursor.fetchall())]

    conn.close()

    print("Importing images...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        future_to_file = {}
        for dir_name in image_dirs:
            dir_path = os.path.join(os.path.dirname(db_path), dir_name)
            if os.path.exists(dir_path):
                files = [f for f in os.listdir(dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
                for filename in files:
                    file_path = os.path.join(dir_path, filename)
                    future = executor.submit(process_image, file_path)
                    future_to_file[future] = filename

        for future in tqdm(concurrent.futures.as_completed(future_to_file), total=len(future_to_file), desc="Processing images"):
            filename = future_to_file[future]
            try:
                data['images'][filename] = future.result()
            except Exception as exc:
                print(f'{filename} generated an exception: {exc}')

    return data

# Usage
db_path = "../visionUI/vhelio_holes/visionUI.db"
image_dirs = ["candidates", "keyframes", "images"]

print("Starting data import...")
imported_data = import_data(db_path, image_dirs)

print(f"\nImport completed:")
print(f"Imported {len(imported_data['annotations'])} annotations")
print(f"Imported {len(imported_data['classes'])} classes")
print(f"Imported {len(imported_data['images'])} images")

Starting data import...
Importing classes...


100%|██████████| 2/2 [00:00<00:00, 39568.91it/s]


Importing annotations...


100%|██████████| 1816/1816 [00:00<00:00, 764130.83it/s]


Importing images...


Processing images: 100%|██████████| 1745/1745 [00:01<00:00, 888.17it/s]


Import completed:
Imported 1816 annotations
Imported 2 classes
Imported 1433 images


In [4]:
list(imported_data['images'].keys())[:10]


['2024-07-19_16-19-55.303542.jpg',
 '2024-07-19_16-19-56.304799.jpg',
 '2024-04-22_17-08-41.837425.jpg',
 '2024-07-30_20-13-51.751979.jpg',
 '2024-08-01_14-19-44.842556.jpg',
 '2024-07-30_20-13-57.995781.jpg',
 '2024-04-22_17-07-51.839890.jpg',
 '2024-04-22_17-09-42.345629.jpg',
 '2024-07-30_20-14-55.829561.jpg',
 '2024-05-21_17-56-56.836300.jpg']

In [9]:
from database import DatabaseManager, Image, Annotation
import os
import shutil
from tqdm import tqdm

def import_to_new_database(imported_data, new_db_dir):
    # Ensure the directory for the new database exists
    os.makedirs(new_db_dir, exist_ok=True)
    
    # Create paths for the new database and image directories
    new_db_path = os.path.join(new_db_dir, "dataset.db")
    candidates_dir = os.path.join(new_db_dir, "candidates")
    keyframes_dir = os.path.join(new_db_dir, "keyframes")
    
    # Create image directories
    os.makedirs(candidates_dir, exist_ok=True)
    os.makedirs(keyframes_dir, exist_ok=True)
    
    # Create a new database manager
    new_db = DatabaseManager(new_db_path)

    print("Importing images and annotations...")
    for filename, img_data in tqdm(imported_data['images'].items(), desc="Processing images"):
        # Determine if the image is a candidate or keyframe (you may need to adjust this logic)
        is_keyframe = any(ann['file_id'] == filename for ann in imported_data['annotations'])
        
        # Copy the image to the appropriate directory
        src_path = os.path.join(os.path.dirname(new_db_dir), filename)
        dst_dir = keyframes_dir if is_keyframe else candidates_dir
        dst_path = os.path.join(dst_dir, filename)
        shutil.copy2(src_path, dst_path)
        
        # Add the image to the new database
        relative_path = os.path.relpath(dst_path, new_db_dir)
        new_image = new_db.add_image(
            path=relative_path,
            width=img_data['width'],
            height=img_data['height'],
            is_train=is_keyframe,  # Assuming all keyframes are for training
            is_test=False  # You may want to adjust this based on your needs
        )
        
        if new_image:
            # Find and add annotations for this image
            for ann in imported_data['annotations']:
                if ann['file_id'] == filename:
                    # Determine the annotation type
                    ann_type = 'point' if ann['type'] == 1 else 'rectangle' if ann['type'] == 3 else 'unknown'
                    
                    new_db.add_annotation(
                        image_id=new_image.id,
                        annotation_type=ann_type,
                        x1=ann['x'],
                        y1=ann['y'],
                        x2=ann['x2'] if ann_type == 'rectangle' else None,
                        y2=ann['y2'] if ann_type == 'rectangle' else None,
                        class_name=imported_data['classes'].get(ann['class_id'], {}).get('name', 'unknown')
                    )

    print(f"Import completed. New database created at {new_db_path}")
    return new_db

# Usage
current_dir = os.getcwd()
new_db_dir = os.path.join(current_dir, "test2")

new_db = import_to_new_database(imported_data, new_db_dir)

# Print some statistics about the new database
with new_db.SessionLocal() as session:
    image_count = session.query(Image).count()
    annotation_count = session.query(Annotation).count()
    point_count = session.query(Annotation).filter_by(type='point').count()
    rectangle_count = session.query(Annotation).filter_by(type='rectangle').count()
    candidate_count = session.query(Image).filter_by(is_train=False, is_test=False).count()
    keyframe_count = session.query(Image).filter_by(is_train=True).count()

print(f"New database statistics:")
print(f"Total images: {image_count}")
print(f"Candidate images: {candidate_count}")
print(f"Keyframe images: {keyframe_count}")
print(f"Total annotations: {annotation_count}")
print(f"Point annotations: {point_count}")
print(f"Rectangle annotations: {rectangle_count}")

Importing images and annotations...


Processing images:   0%|          | 0/1433 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/yves/Projects/active/HLA/MUAL/OpenArmVision/visionAI/2024-07-19_16-19-55.303542.jpg'